In [8]:
import pandas as pd
import json
import os
from dotenv import load_dotenv

In [3]:
full_comments = pd.read_csv("data/full_comments.csv")
examples = full_comments.loc[full_comments['true_label'].notna(), ["comment_text", "true_label"]].head(10)
negative_examples = pd.read_excel("negative_examples.xlsx", names=['comment_text', 'original_label', 'corrected_label'])

examples = examples.rename(columns={"comment_text": "comment", "true_label": "label"})

examples_json = json.dumps(examples.to_dict(orient="records"), indent=2)

negative_examples_json = json.dumps(negative_examples.to_dict(orient="records"), indent=2)

In [20]:
subset = full_comments.sample(30000, random_state=10)
subset['label'] = subset['true_label']

In [ ]:
# INPUT_CSV = "data/full_comments.csv"           # Path to your CSV file
COMMENT_COLUMN = "comment_text"           # Column name containing the comment text
OUTPUT_CSV = "comments_labeled.csv"  # Where to save results

In [9]:
load_dotenv()

api_key = os.environ.get('ANTHROPIC_API_KEY')

In [ ]:
"""
Comment Function Labeling Script
=================================
Uses the Anthropic API to classify comments into one of five categories:
Argumentative, Informational, Opinion, Expressive, Neutral

Usage:
    1. Set your API key: export ANTHROPIC_API_KEY="your-key-here"
    2. Update INPUT_CSV and COMMENT_COLUMN below to match your data
    3. Run: python label_comments.py

Requirements:
    pip install anthropic pandas
"""

import os
import re
import json
import ast
import time
from anthropic import Anthropic

# ============================================================
# CONFIGURATION — Update these to match your data
# ============================================================

MODEL = "claude-haiku-4-5-20251001"  # Fast and cheap, great for classification
BATCH_SIZE = 50                      # Comments per API call
SAVE_EVERY = 2                       # Save progress every N batches

# ============================================================
# LABEL DEFINITIONS (from your labels.md)
# ============================================================
SYSTEM_PROMPT = f"""You are a comment classifier. You will be given a batch of comments, each with an ID number. 
Classify each comment into exactly ONE of these five categories:

**Argumentative**
- Makes specific claims, predictions, or assertions supported by reasoning
- Uses evidence, anecdotes, or scenarios to build a case
- The key distinction from Opinion: there's an attempt to *persuade* or *explain why*, not just state a position

**Informational**
- Shares facts, data, links, or context relevant to the discussion
- Low emotional affect — the comment is trying to *inform*, not convince or react
- Includes answering another commenter's question with factual content
- The key distinction from Argumentative: presenting information without advocating for a position

**Opinion**
- States a value judgment, stance, or take without substantial reasoning
- "This is good/bad/wrong/overrated" — the comment *asserts* but doesn't *argue*
- The key distinction from Argumentative: no real attempt to persuade or support the claim
- The key distinction from Expressive: the comment is making a point, not just reacting

**Expressive**
- Emotional reactions, sarcasm, jokes, venting, exclamations
- The comment is primarily *expressing feeling* rather than making a point
- Includes performative agreement/disagreement ("THIS," "lol exactly," "what a joke")
- The key distinction from Opinion: no identifiable stance being taken, just affect

**Neutral**
- Clarifying or rhetorical questions, meta-commentary, off-topic remarks
- Comments that don't clearly fit the other four categories
- Includes simple factual questions directed at other commenters

**Correctly labeled examples** — these demonstrate the correct label for each comment:
{examples_json}

**Incorrectly labeled examples** — these were originally mislabeled. The "original_label" is the wrong label that was assigned, and the "corrected_label" is what the label should have been. Use these to understand common mistakes to avoid:
{negative_examples_json}

Respond with ONLY a valid JSON array where each element has "id", "label" keys and a confidence indicator where 
0 is not confident in the chosen label and 1 is confident in the chosen label.
Example: [{{"id": 0, "label": "Argumentative", "confidence": 1}}, {{"id": 1, "label": "Expressive", "confidence": 0}}]

Do not include any text outside the JSON array. No explanations, no markdown."""


VALID_LABELS = {"Argumentative", "Informational", "Opinion", "Expressive", "Neutral"}


def format_batch(comments: list[tuple[int, str]]) -> str:
    """Format a batch of (id, comment) tuples into the user message."""
    lines = []
    for idx, comment in comments:
        # Truncate very long comments to avoid token waste
        truncated = comment[:1500] if len(comment) > 1500 else comment
        lines.append(f"[{idx}] {truncated}")
    return "\n\n".join(lines)


def save_results(df):
    """Concat new results with existing OUTPUT_CSV if it exists, then save."""
    if os.path.exists(OUTPUT_CSV):
        existing = pd.read_csv(OUTPUT_CSV)
        combined = pd.concat([existing, df], ignore_index=True)
    else:
        combined = df
    combined.to_csv(OUTPUT_CSV, index=False)
    return combined


def parse_response(response_text: str, expected_ids: list[int]) -> dict[int, str]:
    """Parse the API response JSON into a dict of {id: label}."""
    text = response_text.strip()

    # Strip markdown code fences
    if text.startswith("```"):
        text = text.split("\n", 1)[1]
        text = text.rsplit("```", 1)[0]

    # Try standard JSON first
    try:
        results = json.loads(text)
    except json.JSONDecodeError:
        # Model may return Python-style output (single quotes) instead of JSON
        try:
            results = ast.literal_eval(text)
        except (ValueError, SyntaxError):
            # Last resort: find the first JSON array in the text
            match = re.search(r'\[.*\]', text, re.DOTALL)
            if match:
                try:
                    results = json.loads(match.group())
                except json.JSONDecodeError:
                    print(f"  WARNING: Failed to parse response")
                    return {}
            else:
                print(f"  WARNING: No JSON array found in response")
                return {}

    # Unwrap nested list if model double-wrapped the array
    if results and isinstance(results[0], list):
        results = results[0]

    labels = {}
    for item in results:
        idx = item.get("id")
        label = item.get("label", "").strip()
        conf = item.get('confidence', "")
        
        # Validate
        if idx not in expected_ids:
            print(f"  WARNING: Unexpected ID {idx} in response")
            continue
        if label not in VALID_LABELS:
            # Try case-insensitive match
            matched = [v for v in VALID_LABELS if v.lower() == label.lower()]
            if matched:
                label = matched[0]
            else:
                print(f"  WARNING: Invalid label '{label}' for ID {idx}")
                continue
        
        labels[idx] = {"label": label, "confidence": conf}
    
    return labels


def label_batch(client: Anthropic, comments: list[tuple[int, str]], max_retries: int = 3) -> dict[int, str]:
    """Send a batch to the API and return labels. Retries on failure."""
    user_message = format_batch(comments)
    expected_ids = [idx for idx, _ in comments]
    
    for attempt in range(max_retries):
        try:
            response = client.messages.create(
                model=MODEL,
                max_tokens=1024,
                system=SYSTEM_PROMPT,
                messages=[{"role": "user", "content": user_message}]
            )
            
            response_text = response.content[0].text
            labels = parse_response(response_text, expected_ids)
            
            # Check if we got all expected labels
            missing = set(expected_ids) - set(labels.keys())
            if missing:
                print(f"  WARNING: Missing labels for IDs {missing}")
                if attempt < max_retries - 1:
                    print(f"  Retrying... (attempt {attempt + 2}/{max_retries})")
                    time.sleep(1)
                    continue
            
            return labels
            
        except Exception as e:
            print(f"  ERROR: {e}")
            if attempt < max_retries - 1:
                wait = 2 ** attempt  # Exponential backoff
                print(f"  Retrying in {wait}s... (attempt {attempt + 2}/{max_retries})")
                time.sleep(wait)
            else:
                print(f"  Failed after {max_retries} attempts")
                return {}
    
    return {}


def main():
    df = subset.copy()
    print(f"  {len(df)} rows loaded")
    
    if COMMENT_COLUMN not in df.columns:
        print(f"ERROR: Column '{COMMENT_COLUMN}' not found. Available columns: {list(df.columns)}")
        return
    
    # Skip rows that already have labels (for resuming interrupted runs)
    if "label" not in df.columns:
        df["label"] = None
    
    unlabeled_mask = df["label"].isna()
    unlabeled_indices = df[unlabeled_mask].index.tolist()
    print(f"  {len(unlabeled_indices)} comments to label ({len(df) - len(unlabeled_indices)} already labeled)")
    
    if not unlabeled_indices:
        print("All comments already labeled!")
        return
    
    # Create batches
    batches = []
    for i in range(0, len(unlabeled_indices), BATCH_SIZE):
        batch_indices = unlabeled_indices[i:i + BATCH_SIZE]
        batch = [(idx, str(df.loc[idx, COMMENT_COLUMN])) for idx in batch_indices]
        batches.append(batch)
    
    print(f"  {len(batches)} batches of ~{BATCH_SIZE} comments each\n")
    
    # Process
    client = Anthropic(api_key=api_key)
    total_labeled = 0
    
    for batch_num, batch in enumerate(batches, 1):
        print(f"Batch {batch_num}/{len(batches)} ({len(batch)} comments)...", end=" ")
        
        labels = label_batch(client, batch)
        
        # Write labels to dataframe
        for idx, value in labels.items():
            df.loc[idx, "label"] = value['label']
            df.loc[idx, "confidence"] = value['confidence']
        
        total_labeled += len(labels)
        print(f"got {len(labels)} labels (total: {total_labeled}/{len(unlabeled_indices)})")
        
        # Periodic save — concat with existing file
        if batch_num % SAVE_EVERY == 0:
            combined = save_results(df)
            print(f"  >> Progress saved to {OUTPUT_CSV} ({len(combined)} total rows)")
        
        # Small delay to stay well within rate limits
        if batch_num < len(batches):
            time.sleep(0.5)
    
    # Final save — concat with existing file
    combined = save_results(df)
    
    # Summary
    print(f"\n{'='*50}")
    print(f"DONE — {total_labeled} comments labeled")
    print(f"Saved to: {OUTPUT_CSV} ({len(combined)} total rows)")
    print(f"\nLabel distribution:")
    print(df["label"].value_counts().to_string())
    
    # Check for any still-unlabeled
    still_missing = df["label"].isna().sum()
    if still_missing:
        print(f"\nWARNING: {still_missing} comments still unlabeled (API failures)")


# if __name__ == "__main__":
#     main()

In [7]:
main()

  101 rows loaded
  101 comments to label (0 already labeled)
  4 batches of ~30 comments each

Batch 1/4 (30 comments)... got 30 labels (total: 30/101)
Batch 2/4 (30 comments)... got 30 labels (total: 60/101)
Batch 3/4 (30 comments)... got 30 labels (total: 90/101)
Batch 4/4 (11 comments)... got 11 labels (total: 101/101)

DONE — 101 comments labeled
Saved to: comments_labeled.csv (5103 total rows)

Label distribution:
label
Expressive       28
Opinion          26
Neutral          17
Argumentative    16
Informational    14


In [ ]:
new_df = pd.read_csv("comments_labeled.csv")
new_df.shape

(5103, 5)

In [50]:
comments_labeled = pd.concat([new_df, new_df1])